In [15]:
import pandas as pd

In [16]:
years = [1,2,3,4]
indirect_rates = {'UOG': 0.15, 'CEMML': 0.26, 'NCSU': 0.276}
institutions = ['CEMML','NCSU','UOG']

# indirect rates are applied only to direct_cost_types
direct_cost_types = ['labor', 'materials', 'travel']

cost_types = ['labor', 'materials', 'travel', 'rent',  'indirects']

In [17]:
# Import the CSV file into a pandas data frame 

df = pd.read_csv('SERDP-Budget.csv')
df['total'] = df.qty_or_fte * df.unit_cost

In [18]:
# Check the data

bad_data = False
for i,r in df.iterrows():
    if r.year not in years:
        bad_data = True
        print('ERROR: Bad year field in record {}'.format(i))
        print(df.iloc[i,:])
        print
    if r.institution not in institutions:
        bad_data = True
        print('ERROR: Bad institution field in record {}'.format(i))
        print(df.iloc[i,:])
        print
    if r.cost_type not in cost_types:
        bad_data = True
        print('ERROR: Bad cost_type field in record {}'.format(i))
        print(df.iloc[i,:])
        print 
        
# Stop processing if bad_data

if bad_data:
    print('Did not generate budget-table.tex because data errors were detected.')
    assert False

In [19]:
# Create a dict containing costs sumed over year, institution and cost_type

d = df.groupby(['year', 'institution', 'cost_type'])['total'].sum().to_dict()
#d

In [20]:
# Create a dict using a tuple as a key: (year, institution, cost_type)
# mydict([2, 'UOG', materials']) returns total materials cost for UOG in year 2

mydict= {}
for year in years:
    for institution in institutions:
        for cost_type in cost_types:
            cost = int(d.get((year, institution, cost_type), 0))
            mydict[(year, institution, cost_type)] = cost
#mydict

In [21]:
# Calculate indirects for each year and store in mydict

for year in years:  
    for institution in institutions:
        year_total = 0
        for cost_type in direct_cost_types:
            year_total += mydict[(year, institution, cost_type)]
        indirects = indirect_rates[institution] * year_total
        mydict[(year, institution, 'indirects')] = int(indirects)
        
        # Increase NCS indirects for subcontracts
        # NCS applies indirect charges to the first $25k of each subcontract
        # Here we have subcontracts for UOG and CEMML, both >$25k
        if institution == 'NCSU':
            mydict[(year, 'NCSU', 'indirects' )] += indirect_rates['NCSU'] * 50000
#mydict

In [22]:
# Calculate year totals for all institutions and store in mydict

for year in years:
    for cost_type in cost_types:
        year_total = 0
        for institution in institutions:
            year_total += mydict[(year, institution, cost_type)]
        mydict[year, 'All', cost_type] = int(year_total)
#mydict

In [23]:
# Generate budget-table.tex

tex=r"""
\begin{tabular}{llrrrrrr}
\toprule
Year & Institution & Labor & Materials & Travel & Rent & Indirects & Year total \\
\midrule
"""
for year in years:
    for institution in institutions:
        labor= mydict[(year, institution, 'labor')]
        materials = mydict[(year, institution, 'materials')]
        travel = mydict[(year, institution, 'travel')]
        rent = mydict[(year, institution, 'rent')]
        indirects = mydict[(year, institution, 'indirects')]
        year_total = labor + materials + travel + rent + indirects
        tex += '{} & {} & \${:0,.0f} & \${:0,.0f} & \${:0,.0f} & \${:0,.0f} & \${:0,.0f} & \${:0,.0f} \\\\ \n'.format(
            year, institution, labor, materials, travel, rent, indirects, year_total)  
    tex += r"""
    \midrule    
    """
    labor= mydict[(year, 'All', 'labor')]
    materials = mydict[(year, 'All', 'materials')]
    travel = mydict[(year, 'All', 'travel')]
    rent = mydict[(year, 'All', 'rent')]
    indirects = mydict[(year, 'All', 'indirects')]
    year_total = labor + materials + travel + rent + indirects
    tex += '{} & {} & \${:0,.0f} & \${:0,.0f} & \${:0,.0f} & \${:0,.0f} & \${:0,.0f} & \${:0,.0f} \\\\ \n'.format(
        year, 'All', labor, materials, travel, rent, indirects, year_total) 
    tex += r"""
    \midrule
    """
    
tex += r"""
\bottomrule
\end{tabular}
"""
with open('budget-table.tex', 'w')as f:
    f.write(tex)

In [24]:
print('budget-table.tex has been generated.')

budget-table.tex has been generated.


In [25]:
#pd.set_option('precision', 2)
#df.to_latex('test.tex', index=False, column_format='p{1in}lllllp{1in}r')

In [26]:
df.sort_values(by=['year','cost_type','institution','cost_type'], inplace=True)
df.head()

,cost_item,cost_type,institution,year,qty_or_fte,unit_cost,units,total
2,CEMML Field Admin Support,labor,CEMML,1,0.20,90540.0,$/year (inc. benefits),18108.0
3,CEMML survey tech 1,labor,CEMML,1,1.00,60360.0,$/year (inc. benefits),60360.0
4,CEMML survey tech 2,labor,CEMML,1,1.00,60360.0,$/year (inc. benefits),60360.0
5,CEMML entomologist,labor,CEMML,1,0.05,135810.0,$/year (inc. benefits),6790.5
6,NCSU-PI,labor,NCSU,1,0.05,83000.0,$/year (inc. benefits),4150.0


In [27]:
for institution in institutions:
    for year in years:
        tex=r"""
%\documentclass[11pt,english,letterpaper]{scrartcl}
%\usepackage{booktabs}
%\usepackage[top=1in, bottom=1in, left=1in, right=1in]{geometry}
%\begin{document}
\begin{tabular}{p{1.5in}lrrp{1.5in}r}
\toprule
cost_item & cost_type & qty_or_fte & unit_cost & units & total \\
\midrule
"""
        df1 =  df[df['institution']==institution][df['year']==year]
        for i,r in df1.iterrows():
            tex += '{} & {} & {:0,.2f} & ${:0,.2f} & {} & ${:0,.0f} \\\\ \n'.format(r. cost_item,
                                                                r.cost_type,
                                                                r.qty_or_fte,
                                                                r.unit_cost,
                                                                r.units,
                                                                r.total)
            tex += '\\midrule \n'
        filename = '{}-y{}.tex'.format(institution, year)
        tex += r"""
\bottomrule
\end{tabular}
%\end{document}
"""
        tex = tex.replace('$', '\$')
        tex = tex.replace('_', '\_')
        with open(filename, 'w')as f:
            f.write(tex)

/home/aubrey/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


In [28]:
df

,cost_item,cost_type,institution,year,qty_or_fte,unit_cost,units,total
2,CEMML Field Admin Support,labor,CEMML,1,0.20,90540.00,$/year (inc. benefits),18108.00
3,CEMML survey tech 1,labor,CEMML,1,1.00,60360.00,$/year (inc. benefits),60360.00
4,CEMML survey tech 2,labor,CEMML,1,1.00,60360.00,$/year (inc. benefits),60360.00
5,CEMML entomologist,labor,CEMML,1,0.05,135810.00,$/year (inc. benefits),6790.50
6,NCSU-PI,labor,NCSU,1,0.05,83000.00,$/year (inc. benefits),4150.00
28,NCSU-SME,labor,NCSU,1,1040.00,36.50,$/per hour,37960.00
11,UOG lab tech 1,labor,UOG,1,1.00,60360.00,$/per year (inc. benefits),60360.00
12,UOG lab tech 2,labor,UOG,1,1.00,60360.00,$/per year (inc. benefits),60360.00
13,UOG survey tech 1,labor,UOG,1,1.00,60360.00,$/per year (inc. benefits),60360.00
14,UOG survey tech 2,labor,UOG,1,1.00,60360.00,$/per year (inc. benefits),60360.00
